# one notebook

notebooks are literate programs encoded in json. as literate programs, they concurrently represent both code and formal documentation. a single notebook can be through of as a program, a test, or documentation.

`dgaf` configures complete development environments for notebooks.

> one blog post.

In [9]:
import pytest
@pytest.fixture
def project(pytester):
    import nbformat, dgaf
    name = "sample_notebook_project"
    pytester.makefile(".ipynb", **{name: nbformat.v4.writes(
        nbformat.v4.new_notebook(cells=[
            nbformat.v4.new_markdown_cell("a sample notebook for testing."),
            nbformat.v4.new_code_cell("import pandas"),
        ])
    )})
    project = dgaf.Project(pytester.path)
    yield project

In [1]:
def test_project(project):
    import dgaf
    assert project.fs.files
    assert project.get_name() == "sample_notebook_project"
    assert "nbval" in project.get_test_requires()
    assert not project.get_url()
    assert project.get_requires() == ["pandas"]

In [1]:
def test_flit(pytester, project):
    # when there is no url we can initialize one in a few ways.
    import dgaf, shutil
    pyproject = project/dgaf.PYPROJECT_TOML
    assert not pyproject.exists()
    project.to_flit()
    assert pyproject.exists()
    result = pytester.run(*"flit build".split())
    assert not result.ret

In [1]:
def test_setuptools(pytester, project):
    import dgaf
    setup = project/dgaf.SETUP_CFG
    assert not setup.exists()
    project.to_setuptools()
    assert setup.exists()
    project.to_setup_py()
    result = pytester.run(*"python setup.py build --dry-run".split())
    assert not result.ret

A single notebook, markdown file, or rst document may represent a post in the file name conforms to the `"YYYY-MM-DD-title.ext"` pattern.